In [1]:
# %matplotlib widget

import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy import units as u
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, get_masters_metadata

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.{ext}")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.{ext}")

# overwrite or not
OVERWRITE = False

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]

# get master calibration metadata
masters_table = get_masters_metadata(path_pattern=out_calib_path)

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [4]:
# define list of arc exposures
# expnum_list = list(range(685,691))

arc_analogs = frames_table.query("mjd>=60039 & (imagetyp=='arc'|imagetyp=='object') & (neon|hgne) & not (argon|ldls|xenon)").groupby(["mjd", "camera", "exptime"])

for mjd, camera, exptime in arc_analogs.groups:
    arcs = arc_analogs.get_group((mjd, camera, exptime))
    print(arcs.to_string())
    
    master_bias = masters_table.query("mjd=='super' & kind=='mbias' & camera==@camera")
    master_dark = masters_table.query("mjd=='super' & kind=='mdark' & camera==@camera").sort_values("exptime")
    try:
        mbias = master_bias.iloc[0]
    except IndexError:
        imageMethod.image_logger.error(f"no master bias found for {mjd = }, {camera = }, {exptime = }")
    try:
        mdark = master_dark.loc[master_dark.exptime>=exptime].iloc[0]
    except IndexError:
        imageMethod.image_logger.error(f"no master dark found for {mjd = }, {camera = }, {exptime = }")
        continue
    
    # let's pre-process the all arc frames
    carc_paths = []
    for _, arc_frame in arcs.iterrows():
        arc_path = arc_frame["path"]
        arc_expnum = arc_frame["expnum"]

        carc_path = out_main_path.format(kind="carc", camera=camera, expnum=arc_expnum, ext="fits")
        carc_paths.append(carc_path)
        if OVERWRITE:
            imageMethod.preprocRawFrame_drp(
                in_image=arc_path,
                out_image=out_main_path.format(kind="parc", camera=camera, expnum=arc_expnum, ext="fits"),
                subtract_overscan=True, unit="electron", compute_error=True,
                os_bound_y="1,4080", os_bound_x="2044,2077", assume_imagetyp="arc"
            )
            imageMethod.basicCalibration_drp(
                in_image=out_main_path.format(kind="parc", camera=camera, expnum=arc_expnum, ext="fits"),
                out_image=carc_path,
                in_bias=mbias.path,
                in_dark=mdark.path
            )

    imageMethod.createMasterFrame_drp(
        in_images=carc_paths,
        out_image=out_calib_path.format(mjd=mjd, kind="marc", camera=camera, exptime=int(exptime), ext="fits"),
    )

    imagetyp spec    mjd camera    expnum  exptime  argon   neon   ldls  hgne  xenon                                                                                                                      path
306      arc  sp1  60039     b1  00000535     10.0  False  False  False  True  False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000535.fits.gz
[WARNING]: building master frame with one image file
[INFO]: input frames: ./data/lvm-carc-b1-00000535.fits
[INFO]: combining 1 frames into master frame
[INFO]: updating header for new master frame './data/lvm-60039-marc-b1-10.fits'
    imagetyp spec    mjd camera    expnum  exptime  argon   neon   ldls  hgne  xenon                                                                                                                      path
450      arc  sp1  60039     b1  00000538    270.0  False  False  False  True  False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_c

In [5]:
# define list of arc exposures
# expnum_list = list(range(685,691))

arc_analogs = frames_table.query("mjd>=60039 & (imagetyp=='arc'|imagetyp=='object') & (neon|hgne) & not (argon|ldls|xenon)").groupby("camera")

for camera in arc_analogs.groups:
    arcs = arc_analogs.get_group(camera)
    print(arcs.to_string())
    
    master_bias = masters_table.query("mjd=='super' & kind=='mbias' & camera==@camera")
    master_dark = masters_table.query("mjd=='super' & kind=='mdark' & camera==@camera").sort_values("exptime")
    try:
        mbias = master_bias.iloc[0]
    except IndexError:
        imageMethod.image_logger.error(f"no master bias found for {camera = }")
    try:
        mdark = master_dark.loc[master_dark.exptime>=exptime].iloc[0]
    except IndexError:
        imageMethod.image_logger.error(f"no master dark found for {camera = }")
        continue
    
    # let's pre-process the all arc frames
    carc_paths = []
    for _, arc_frame in arcs.iterrows():
        arc_path = arc_frame["path"]
        arc_expnum = arc_frame["expnum"]

        carc_path = out_main_path.format(kind="carc", camera=camera, expnum=arc_expnum, ext="fits")
        carc_paths.append(carc_path)

    imageMethod.createMasterFrame_drp(
        in_images=carc_paths,
        out_image=out_calib_path.format(mjd="super", kind="marc", camera=camera, exptime="x", ext="fits"),
    )

     imagetyp spec    mjd camera    expnum  exptime  argon   neon   ldls   hgne  xenon                                                                                                                      path
306       arc  sp1  60039     b1  00000535     10.0  False  False  False   True  False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000535.fits.gz
450       arc  sp1  60039     b1  00000538    270.0  False  False  False   True  False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000538.fits.gz
869    object  sp1  60041     b1  00000688     30.0  False  False  False   True  False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60041/sdR-s-b1-00000688.fits.gz
979    object  sp1  60041     b1  00000689     90.0  False  False  False   True  False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/

In [7]:
master_arcs = get_masters_metadata(path_pattern=out_calib_path, kind="marc").sort_values(["camera", "exptime"])
print(master_arcs.to_string())

for _, marc in master_arcs.iterrows():
    print(marc)
    master_trace = masters_table.query("mjd=='super' & kind=='mtrace' & camera==@marc.camera & exptime=='x'")
    try:
        mtrace = master_trace.iloc[0]
    except:
        imageMethod.image_logger.error(f"no master trace found for '{marc.path}'")
        continue
    
    imageMethod.extractSpec_drp(
        in_image=marc.path,
        out_rss=out_calib_path.format(mjd=marc.mjd, kind="xarc", camera=marc.camera, exptime=int(marc.exptime) if marc.exptime != "x" else marc.exptime, ext="fits"),
        in_trace=mtrace.path,
        method="aperture", aperture=3, plot=1, parallel="auto"
    )

       mjd  kind camera exptime                               path
67   super  marc     b1     3.0    ./data/lvm-super-marc-b1-3.fits
81   60011  marc     b1     3.0    ./data/lvm-60011-marc-b1-3.fits
4    super  marc     b1    10.0   ./data/lvm-super-marc-b1-10.fits
79   60011  marc     b1    10.0   ./data/lvm-60011-marc-b1-10.fits
96   60039  marc     b1    10.0   ./data/lvm-60039-marc-b1-10.fits
53   super  marc     b1    30.0   ./data/lvm-super-marc-b1-30.fits
76   60041  marc     b1    30.0   ./data/lvm-60041-marc-b1-30.fits
95   60011  marc     b1    30.0   ./data/lvm-60011-marc-b1-30.fits
9    super  marc     b1    90.0   ./data/lvm-super-marc-b1-90.fits
10   60041  marc     b1    90.0   ./data/lvm-60041-marc-b1-90.fits
97   60011  marc     b1    90.0   ./data/lvm-60011-marc-b1-90.fits
7    60011  marc     b1   270.0  ./data/lvm-60011-marc-b1-270.fits
26   60041  marc     b1   270.0  ./data/lvm-60041-marc-b1-270.fits
54   60039  marc     b1   270.0  ./data/lvm-60039-marc-b1-270.